In [ ]:
# Setup

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

import torchvision.datasets as datasets
from torchvision import datasets, models, transforms
from torch.autograd import Variable

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
# HyperParameters

num_epochs = 1
batch_size_train = 32
batch_size_test = 32
learning_rate = 0.01
momentum = 0.5

In [ ]:
# Prepare raw dataset
import zipfile
from google.colab import drive

drive_dir = '/content/gdrive'
drive.mount(drive_dir)
work_dir = f'{drive_dir}/My Drive/Reproduction_DL'


!ls "/content/gdrive/My Drive/Reproduction_DL/ISIC"


dataset_list = ['ISBI2016_ISIC_Part1_Test_Data.zip', 'ISBI2016_ISIC_Part1_Test_GroundTruth.zip', 'ISBI2016_ISIC_Part1_Training_Data.zip', 'ISBI2016_ISIC_Part1_Training_GroundTruth.zip']

for d in dataset_list:
  zip_ref = zipfile.ZipFile(f'{work_dir}/ISIC/{d}', 'r')
  # folder_to_extract = d.strip('.zip')
  zip_ref.extractall(f'{work_dir}/ISIC/')
  zip_ref.close()


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ISBI2016_ISIC_Part1_Test_Data
ISBI2016_ISIC_Part1_Test_Data.zip
ISBI2016_ISIC_Part1_Test_GroundTruth
ISBI2016_ISIC_Part1_Test_GroundTruth.zip
ISBI2016_ISIC_Part1_Training_Data
ISBI2016_ISIC_Part1_Training_Data.zip
ISBI2016_ISIC_Part1_Training_GroundTruth
ISBI2016_ISIC_Part1_Training_GroundTruth.zip


In [ ]:
# Load the ISIC dataset (assumes it's already preprocessed)

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.ImageFolder("path/to/train_data", transform=train_transform)
val_dataset = datasets.ImageFolder("path/to/val_data", transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size_test, shuffle=False, num_workers=4)


FileNotFoundError: ignored

In [ ]:
# Load Model

model = models.vgg13(pretrained=True)

# Make sure the input type and the weight type be on the same device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
input_size=(3, 244, 244)
summary(model, input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 244, 244]           1,792
              ReLU-2         [-1, 64, 244, 244]               0
            Conv2d-3         [-1, 64, 244, 244]          36,928
              ReLU-4         [-1, 64, 244, 244]               0
         MaxPool2d-5         [-1, 64, 122, 122]               0
            Conv2d-6        [-1, 128, 122, 122]          73,856
              ReLU-7        [-1, 128, 122, 122]               0
            Conv2d-8        [-1, 128, 122, 122]         147,584
              ReLU-9        [-1, 128, 122, 122]               0
        MaxPool2d-10          [-1, 128, 61, 61]               0
           Conv2d-11          [-1, 256, 61, 61]         295,168
             ReLU-12          [-1, 256, 61, 61]               0
           Conv2d-13          [-1, 256, 61, 61]         590,080
             ReLU-14          [-1, 256,

In [ ]:
# Initialize VGG-13 model
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, len(train_dataset.classes))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

NameError: ignored

In [ ]:
# Set loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    grads = ''

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        loss.backward()

        # Calculate gradients and store them
        grads = param.grad.data.view(-1).clone().cpu() for param in model.parameters() if param.grad is not None
        break

        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")


    # Calculate average gradient for the epoch
    # grads = torch.cat(grads)
    avg_grad = torch.mean(grads).item()
    print(f"Epoch {epoch+1} - Average Gradient:", avg_grad)

NameError: ignored

In [ ]:
def compute_loss_and_gradient(model, data, target):
    # Compute the loss and gradient of the model on a single batch
    model.eval()
    loss_fn = nn.CrossEntropyLoss()
    data, target = data.to(device), target.to(device)
    data.requires_grad_()
    output = model(data)
    loss = loss_fn(output, target)
    loss.backward()
    grad = data.grad.clone().detach()
    return loss.item(), grad

def determine_g(model, data_loader):
    # Determine g_m_c for each sample m and class c in the output layer
    model.eval()
    r_c_list = []
    for batch_idx, (data, target) in enumerate(data_loader):
        _, G_H = compute_loss_and_gradient(model, data, target)
        r_c_list.append(G_H[1:] / G_H[0])
        if batch_idx >= 10:  # Only use first 10 batches for efficiency
            break
    r_c = torch.cat(r_c_list)
    
    index_groups = []
    for j in range(r_c.shape[0]):
        found_group = False
        for i, index_group in enumerate(index_groups):
            if torch.allclose(r_c[j], r_c[index_group[0]]):
                index_groups[i].append(j)
                found_group = True
                break
        if not found_group:
            index_groups.append([j])
    
    g = torch.zeros((len(data_loader.dataset), r_c.shape[0]))
    for batch_idx, (data, target) in enumerate(data_loader):
        data = data.to(device)
        output = model(data)
        prob = F.softmax(output, dim=1)
        for c in range(r_c.shape[0]):
            j = index_groups[batch_idx % len(index_groups)][c]
            g[batch_idx*len(data):(batch_idx+1)*len(data)][c] = prob[:, j] / prob[:, 0]
    
    return g